In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib as mpl
# print(mpl.rcParams.items)
mpl.use('Agg')
mpl.rcParams['text.usetex'] = False
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = ['serif']
mpl.rcParams['font.serif'] = ['Times New Roman']
# mpl.rcParams['font.family'] = ['Times New Roman']
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['savefig.dpi'] = 250
mpl.rcParams['figure.dpi'] = 250
mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['savefig.bbox'] = 'tight'
import matplotlib.pyplot as plt
%matplotlib inline
# print(mpl.rcParams.items)

<!-- ![](./header.png) -->
<img src="./header.png",width=100%>

# The Photometric LSST Astronomical Time-series Classification Challenge (PLAsTiCC): Selection of a performance metric

*Alex Malz (NYU)*, *Renee Hlozek (U. Toronto)*, *Tarek Alam (UCL)*, *Anita Bahmanyar (U. Toronto)*, *Rahul Biswas (U. Stockholm)*, *Rafael Martinez-Galarza (Harvard)*, *Gautham Narayan (STScI)*

We describe and illustrate the process by which a global performance metric was chosen for Photometric LSST Astronomical Time-series Classification Challenge (PLAsTiCC), a Kaggle competition aiming to identify promising transient and variable classifiers for LSST by involving the broader community outside astronomy.

Introduction
============

The metric of this note is for the first version of the Kaggle competition, though there are future plans for an early classification challenge and identification of class-specific metrics for different science goals.  

* The metric must return a single scalar value.
* The metric must be well-defined for non-binary classes.
* The metric must balance diverse science use cases in the presence of heavily nonuniform class prevalence.
* The metric must respect the information content of probabilistic classifications.
* The metric must be able to evaluate deterministic classifications.
* The metric must be interpretable, meaning it gives a more optimal value for "good" mock classifiers and a less optimal value for mock classifiers plagued by anticipated systematic errors; in other words, it must pass basic tests of intuition.
* The metric must be reliable, giving consistent results for different instantiations of the same test case.

The Probabilistic Classification Metric (ProClaM) code used in this exploration of performance metrics is publicly available on [GitHub](https://github.com/aimalz/proclam).

In [ ]:
import proclam
from proclam import *

Data
====

We confirm the behavior of the metrics on mock data with well-understood systematics as well as real data from past classification challenges.

## Mock classifier systematics

* guessing: random classifications across all classes
* uncertain: uniform probabilities across all classes
* perfect: perfectly accurate on all classes
* almost: a slight perturbation of the perfect classifier
* noisy: a large perturbation of the perfect classifier
* tunnel vision: classifies one class well and others randomly
* cruise control: classifies all objects as a single class
* subsumed: consistently misclassifies one class as one other class

In [ ]:
plasticc = {}
plasticc['label'] = 'ProClaM'
plasticc['names'] = []
plasticc['cm'] = {}

In [ ]:
M_classes = 13

In [ ]:
chosen = np.random.randint(0, M_classes)
print(chosen)

In [ ]:
def plot_cm_from_cm(cm, text):
    plt.matshow(cm, vmin=0., vmax=1.)
    plt.title(text)
    plt.xlabel('predicted class')
    plt.ylabel('true class')
    plt.colorbar()
    plt.show()
    plt.close()
    
def wrap_up_classifier(cm, testname, info_dict):
    cm = cm / np.sum(cm, axis=1)[:, np.newaxis]
    plot_cm_from_cm(cm, testname)
    info_dict['names'].append(testname)
    info_dict['cm'][testname] = cm
    return info_dict

### Guessing classifier

Totally random CM

In [ ]:
cm = np.random.uniform(size=(M_classes, M_classes))
plasticc = wrap_up_classifier(cm, 'Guessing', plasticc)

### Uncertain

Totally uniform CM

In [ ]:
cm = np.ones((M_classes, M_classes))
plasticc = wrap_up_classifier(cm, 'Uncertain', plasticc)

### Perfect classifier

Identity matrix CM

In [ ]:
cm = np.eye(M_classes)
plasticc = wrap_up_classifier(cm, 'Perfect', plasticc)

### Almost perfect classifier

Identity matrix CM plus low-amplitude uniform

In [ ]:
cm = np.eye(M_classes) + 0.1 * np.ones((M_classes, M_classes))
plasticc = wrap_up_classifier(cm, 'Almost', plasticc)

### Noisy classifier

Identity matrix CM plus high-amplitude uniform

In [ ]:
cm = np.eye(M_classes) + 0.5 * np.ones((M_classes, M_classes))
plasticc = wrap_up_classifier(cm, 'Noisy', plasticc)

### Tunnel vision classifier

accurate predictions on one class and uniform on others

In [ ]:
cm = np.ones((M_classes, M_classes))
cm = cm * 0.1
cm[:, chosen] = np.zeros((M_classes))[np.newaxis, :]
cm[chosen][chosen] = 1.
plasticc = wrap_up_classifier(cm, 'Tunnel', plasticc)

### Cruise control classifier

always predict one class regardless of true class

In [ ]:
cm = np.ones((M_classes, M_classes))
cm = cm * 0.1
cm[:, chosen] = 1.
plasticc = wrap_up_classifier(cm, 'Cruise', plasticc)

### Subsumed classifiers

Subsumed to: the chosen class is consistently misclassified as a different class

Subsumed from: another class is consistently misclassified as the chosen class

In [ ]:
cm = plasticc['cm']['Almost'].copy()
cm[chosen] = cm[chosen-1]
plasticc = wrap_up_classifier(cm, 'SubsumedTo', plasticc)

In [ ]:
cm = plasticc['cm']['Almost'].copy()
cm[chosen-1] = cm[chosen]
plasticc = wrap_up_classifier(cm, 'SubsumedFrom', plasticc)

## Real classification results

* SNPhotCC \[from Michelle?\]
* \[Ashish's data?\]
* \[Renee's data?\]

*show confusion matrices*

In [ ]:
def make_class_pairs(data_info_dict):
    return zip(data_info_dict['classifications'], data_info_dict['truth_tables'])

def make_file_locs(data_info_dict):
    names = data_info_dict['names']
    data_info_dict['dirname'] = topdir + data_info_dict['label'] + '/'
    data_info_dict['classifications'] = ['%s/predicted_prob_%s.csv'%(name, name) for name in names]
    data_info_dict['truth_tables'] = ['%s/truth_table_%s.csv'%(name, name) for name in names]
#     print(data_info_dict)
    return data_info_dict

In [ ]:
mystery = {}
mystery['label'] = 'Unknown'
mystery['names'] = ['RandomForest', 'KNeighbors', 'MLPNeuralNet']

In [ ]:
snphotcc = {}
snphotcc['label'] = 'SNPhotCC'
prefixes = ['Templates', 'Wavelets']
suffixes = ['BoostForest', 'KNN', 'NB', 'NeuralNetwork', 'SVM']
snphotcc['names'] = []
for prefix in prefixes:
    for suffix in suffixes:
        snphotcc['names'].append(prefix+suffix)

In [ ]:
topdir = '../examples/'
for dataset in [mystery, snphotcc]:
    dataset = make_file_locs(dataset)
    dataset['class_pairs'] = make_class_pairs(dataset)

In [ ]:
def plot_cm(probs, truth, text, loc=''):
    cm = proclam.metrics.util.prob_to_cm(probs, truth)
    plt.matshow(cm.T, vmin=0., vmax=1.)
# plt.xticks(range(max(truth)+1), names)
# plt.yticks(range(max(truth)+1), names)
    plt.xlabel('predicted class')
    plt.ylabel('true class')
    plt.colorbar()
    plt.title(text)
    plt.show()
#     plt.savefig(loc+name+'_cm.png')
    plt.close()

In [ ]:
def process_strings(dataset, cc):
    loc = dataset['dirname']
    text = dataset['label'] + ' ' + dataset['names'][cc]
    return loc, text

def just_read_class_pairs(pair, dataset, cc):
    loc, text = process_strings(dataset, cc)
    clfile = pair[0]
    truthfile = pair[1]
    prob_mat = pd.read_csv(loc + clfile, delim_whitespace=True).values
    nobj = np.shape(prob_mat)[0]
    nclass = np.shape(prob_mat)[1]
    truth_values = pd.read_csv(loc + truthfile, delim_whitespace=True).values
    nobj_truth = np.shape(truth_values)[0]
    nclass_truth = np.shape(truth_values)[1]
    tvec = np.where(truth_values==1)[1]
    pmat = prob_mat
    return pmat, tvec
    
def read_class_pairs(pair, dataset, cc):
    fileloc, text = process_strings(dataset, cc)
    pmat, tvec = just_read_class_pairs(pair, dataset, cc)
    plot_cm(pmat, tvec, text, loc=fileloc + dataset['names'][cc] + '/')
    return pmat, tvec

def just_plot_cm(dataset, cc, pmat, tvec):
    fileloc, text = process_strings(dataset, cc)
    plot_cm(pmat, tvec, text, loc=fileloc + dataset['names'][cc] + '/')
    return

Methods (Metrics)
======

We considered two metrics of classification probabilities, each of which is interpretable and avoids reducing probabilities to point estimates

The Brier score is defined as
\begin{eqnarray*}
B &=& \sum_{m=1}^{M}\frac{w_{m}}{N_{m}}\sum_{n=1}^{N_{m}}\left((1-p_{n}(m | m))^{2}+\sum_{m'\neq m}^{M}(p_{n}(m' | m))^{2}\right)
\end{eqnarray*}

The log-loss is defined as
\begin{eqnarray*}
L &=& -\sum_{m=1}^{M}\frac{w_{m}}{N_{m}}\sum_{n=1}^{N_{m}}\ln[p_{n}(m | m)]
\end{eqnarray*}

We calculate the metric within each class by taking an average of its value for each true member of the class.  Then we weight the metrics for each class by an arbitrary weight vector and take a weighted average of the per-class metrics to produce a global scalar metric.

In [ ]:
metricslist = ['Brier', 'LogLoss']
colors = ['b', 'r']
markerlist = ['o', 's', '*']

### Weights

We may take weighted averages of the per-class metrics, and these weights may be considered in terms of the systematics we discussed, by upweighting or downweighting the "chosen" class most affected by the systematics.

In [ ]:
flat_weight = np.ones(M_classes)
hi_weight = np.ones(M_classes) / np.float(M_classes)
hi_weight[chosen] = 1.
lo_weight = np.ones(M_classes) 
lo_weight[chosen] = 1. / np.float(M_classes)
all_weights = {}
all_weights['flat'] = flat_weight
all_weights['up'] = hi_weight
all_weights['down'] = lo_weight
all_weights['per_class'] = 'per_class'
all_weights['per_item'] = 'per_item'

Results
=======

*one plot per set of "true" classes: classifiers on x axis, metrics on y axes*

In [ ]:
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)
        
def per_metric_helper(ax, n, data, metric_names, codes, shapes, colors):
    plot_n = n+1
    in_x = np.arange(len(codes))
    ax_n = ax
    n_factor = 0.1 * (plot_n - 2)
    if plot_n>1:
        ax_n = ax.twinx()
        rot_ang = 270
        label_space = 15.
    else:
        rot_ang = 90
        label_space = 0.
    if plot_n>2:
        ax_n.spines["right"].set_position(("axes", 1. + 0.1 * (plot_n-1)))
        make_patch_spines_invisible(ax_n)
        ax_n.spines["right"].set_visible(True)
    handle = ax_n.scatter(in_x+n_factor*np.ones_like(data[n]), data[n], marker=shapes[n], s=10, color=colors[n], label=metric_names[n])
    ax_n.set_ylabel(metric_names[n], rotation=rot_ang, fontsize=14, labelpad=label_space)
#     ax_n.set_ylim(0.9 * min(data[n]), 1.1 * max(data[n]))
    return(ax, ax_n, handle)

def metric_plot(dataset, res, metric_names, shapes, colors, modtext=''):
    codes = dataset['names']
    data = res
    text = dataset['label']
#     fileloc = dataset['dirname']+dataset['label']+'_results.png'
    xs = np.arange(len(codes))
    fig, ax = plt.subplots()
    fig.subplots_adjust(right=1.)
    handles = []
    for n in range(len(metric_names)):
#         print(np.shape(data[n]))
        (ax, ax_n, handle) = per_metric_helper(ax, n, data, metric_names, codes, shapes, colors)
        handles.append(handle)
    plt.xticks(xs, codes)
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
    plt.xlabel('Classifiers', fontsize=14)
    plt.legend(handles, metric_names, loc='lower left')
    fig.suptitle(text+modtext)
#     plt.savefig(fileloc)
    return

## Mock classifier systematics

In [ ]:
generator = proclam.simulators.LogUnbalanced()
N_objects = 10000
truth = generator.simulate(M_classes, N_objects)

In [ ]:
d = np.diff(np.unique(truth)).min()
left_of_first_bin = truth.min() - float(d)/2
right_of_last_bin = truth.max() + float(d)/2
plt.hist(truth, np.arange(left_of_first_bin, right_of_last_bin + d, d), log=True, alpha=0.5)
# plt.xticks(range(max(truth)+1), names)
plt.hist(truth, log=True, alpha=0.5)
plt.ylabel('counts')
plt.xlabel('class')
plt.show()
plt.close()

In [ ]:
# data = np.empty((len(metricslist), len(plasticc['names'])))
plasticc['probs'] = {}
for cc, name in enumerate(plasticc['names']):
    code = proclam.classifiers.FromCM()
    probs = code.classify(plasticc['cm'][name], truth, other=False)
    plasticc['probs'][name] = probs
#     for count, metric in enumerate(metricslist):
#         D = getattr(proclam.metrics, metric)()
#         hm = D.evaluate(probs, truth, averaging='per_class')
#         data[count][cc] = hm
#     plasticc['probs'] = data

In [ ]:
for wt in all_weights.keys():
    data = np.empty((len(metricslist), len(plasticc['names'])))
    for cc, name in enumerate(plasticc['names']):
        probs = plasticc['probs'][name]
        for count, metric in enumerate(metricslist):
            D = getattr(proclam.metrics, metric)()
            hm = D.evaluate(probs, truth, averaging=all_weights[wt])
            data[count][cc] = hm
#     plasticc['results'] = data
    metric_plot(plasticc, data, metricslist, markerlist, colors, modtext=' '+wt+'weight')

Would like to do this many times to generate error bars

Try with different weights relative to randomly chosen class

## Real classification results

In [ ]:
for dataset in [mystery, snphotcc]:
    data = np.empty((len(metricslist), len(dataset['names'])))
    for cc, pair in enumerate(dataset['class_pairs']):
        probm, truthv = read_class_pairs(pair, dataset, cc)
        for count, metric in enumerate(metricslist):
            D = getattr(proclam.metrics, metric)()
            hm = D.evaluate(probm, truthv)
            data[count][cc] = hm
#     dataset['results'] = data
    metric_plot(dataset, data, metricslist, markerlist, colors)

In [ ]:
# metric_plot(snphotcc, metricslist, markerlist, colors)

In [ ]:
# metric_plot(mystery, metricslist, markerlist, colors)

Conclusions
===========

We conclude that the Brier and log-loss metrics convey different information but are more or less consistent with our intuition for what makes a good classifier.  The Brier metric includes a penalty term not present in the log-loss but somehow is always consistent with the log-loss, meaning the penalty term doesn't really make a difference.  The log-loss has a larger dynamic range, which seems good but probably isn't that big a deal either.

Acknowledgments
===============

The DESC acknowledges ongoing support from the Institut National de Physique Nucleaire et de Physique des Particules in France; the Science & Technology Facilities Council in the United Kingdom; and the Department of Energy, the National Science Foundation, and the LSST Corporation in the United States.

DESC uses resources of the IN2P3 Computing Center (CC-IN2P3--Lyon/Villeurbanne - France) funded by the Centre National de la Recherche Scientifique; the National Energy Research Scientific Computing Center, a DOE Office of Science User Facility supported by the Office of Science of the U.S. Department of Energy under Contract No. DE-AC02-05CH11231; STFC DiRAC HPC Facilities, funded by UK BIS National E-infrastructure capital grants; and the UK particle physics grid, supported by the GridPP Collaboration.

This work was performed in part under DOE Contract DE-AC02-76SF00515.

Contributions
=======

Alex Malz: conceptualization, data curation, formal analysis, investigation, methodology, project administration, software, supervision, validation, visualization, writing - original draft

Renee Hlozek: data curation, formal analysis, funding acquisition, investigation, project administration, software, supervision, validation, visualization, writing - original draft

Tarek Alam: investigation, software, validation

Anita Bahmanyar: formal analysis, investigation, methodology, software, writing - original draft

Rahul Biswas: conceptualization, methodology, software

Rafael Martinez-Galarza: data curation, software, visualization

Gautham Narayan: data curation, formal analysis

In [ ]:
# cells with a tag of "hideme" will not appear in html resulting from:
# jupyter nbconvert desc_note/main.ipynb --TagRemovePreprocessor.remove_cell_tags='["hideme"]'
# jupyter nbconvert desc_note/main.ipynb --TagRemovePreprocessor.remove_input_tags='["hidein"]'
